In [15]:
import os
import string
import sys

import numpy as np
import pandas as pd

In [16]:
def get_fire_zone(fire_number):
    zone = fire_number[0][0:2]
    match zone:
        case "C1":
            return "Cariboo_Quesnel"
        case "C2":
            return "Cariboo_CentralCariboo"
        case "C3":
            return "Cariboo_CentralCariboo"
        case "C4":
            return "Cariboo_100MileHouse"
        case "C5":
            return "Cariboo_Chilcotin"
        case "G1":
            return "PrinceGeorge_PrinceGeorge"
        case "G3":
            return "PrinceGeorge_Robson"
        case "G4":
            return "PrinceGeorge_VanJam"
        case "G5":
            return "PrinceGeorge_VanJam"
        case "G6":
            return "PrinceGeorge_Mackenzie"
        case "G7":
            return "PrinceGeorge_DawsonCreek"
        case "G8":
            return "PrinceGeorge_FortStJohn"
        case "G9":
            return "PrinceGeorge_FortNelson"
        case "K2":
            return "Kamloops_Kamloops"
        case "K4":
            return "Kamloops_Vernon"
        case "K5":
            return "Kamloops_Penticton"
        case "K6":
            return "Kamloops_Merritt"
        case "K7":
            return "Kamloops_Lillooet"
        case "N1":
            return "Southeast_Cranbrook"
        case "N2":
            return "Southeast_Invermere"
        case "N4":
            return "Southeast_Columbia"
        case "N5":
            return "Southeast_Arrow"
        case "N6":
            return "Southeast_Boundary"
        case "N7":
            return "Southeast_Kootenay"
        case "R1":
            return "Northwest_Nadina"
        case "R2":
            return "Northwest_Nadina"
        case "R3":
            return "Northwest_Bulkley"
        case "R4":
            return "Northwest_Bulkley"
        case "R5":
            return "Northwest_Skeena"
        case "R8":
            return "Northwest_Skeena"
        case "R9":
            return "Northwest_Cassiar"
        case "V1":
            return "Coastal_Fraser"
        case "V3":
            return "Coastal_Pemberton"
        case "V5":
            return "Coastal_SunshineCoast"
        case "V6":
            return "Coastal_SouthIsland"
        case "V7":
            return "Coastal_MidIsland"
        case "V8":
            return "Coastal_NorthIslandMidCoast"
        case "V9":
            return "Coastal_NorthIslandMidCoast"
        case "VA":
            return "Coastal_NorthIslandMidCoast"
        case _:
            return np.nan

In [17]:
def get_month(date):
    asnum = date[4]
    asstring = str(asnum)
    shorten = asstring[4:6]
    month = int(shorten)
    #month = int(str(date[4])[5:6])
    match month:
        case 1:
            return "January"
        case 2:
            return "February"
        case 3:
            return "March"
        case 4:
            return "April"
        case 5:
            return "May"
        case 6:
            return "June"
        case 7:
            return "July"
        case 8:
            return "August"
        case 9:
            return "September"
        case 10:
            return "October"
        case 11:
            return "November"
        case 12:
            return "December"
        case _:
            raise Exception("**ERROR** number is not a month")

In [18]:
fire_data = pd.read_csv("data/HistoricalFireData/H_FIRE_PLY.csv")
fire_data = fire_data.drop(columns=['VERSION_NO', 'FIRELABEL', 'SOURCE', 'SHAPE', 'FIRE_CAUSE', 'TRACK_DATE', 'LOAD_DATE', 'METHOD', 'FCODE', 'OBJECTID', 'FEAT_LEN', 'X', 'Y', 'AREA_SQM'])
fire_data = fire_data.drop(fire_data[fire_data['FIRE_YEAR'] < 2009].index)

In [19]:
fire_data.insert(loc=0, column="FIRE_ZONE", value=fire_data.apply(get_fire_zone, axis=1))
fire_data = fire_data.dropna()
fire_data.insert(loc=3, column="FIRE_MONTH", value=fire_data.apply(get_month, axis=1))
fire_data = fire_data.drop(columns=['FIRE_NO', 'FIRE_DATE'])

the first problem I ran into here was with the fire zones. I found a map from 2020 that indicates fire zones, although there are unfortunately some past fire zones that are not indicated by this map. I removed all the data from zones that are not indicated by this map from 2020 (this is what the dropna() is doing in the previous cell). I am also hoping that the fire zones have not changed drastically over the past 14 years or the model will not be helpful (if fire zone C1 meant a different area in 2014 than it does in 2023 for example).

In [20]:
fire_data.head()

,FIRE_ZONE,FIRE_YEAR,FIRE_MONTH,SIZE_HA
18327,Northwest_Cassiar,2009,July,113.9
18328,Northwest_Cassiar,2009,July,118.1
18329,Northwest_Cassiar,2009,July,841.5
18330,Northwest_Cassiar,2009,July,185.0
18331,Kamloops_Kamloops,2009,April,49.7


In [21]:
fire_zones = ["Cariboo_Quesnel", "Cariboo_CentralCariboo", "Cariboo_CentralCariboo", 
              "Cariboo_100MileHouse", "Cariboo_Chilcotin", "PrinceGeorge_PrinceGeorge", 
              "PrinceGeorge_Robson", "PrinceGeorge_VanJam", "PrinceGeorge_Mackenzie", 
              "PrinceGeorge_DawsonCreek", "PrinceGeorge_FortStJohn", "PrinceGeorge_FortNelson",
              "Kamloops_Kamloops", "Kamloops_Vernon", "Kamloops_Penticton",
              "Kamloops_Merritt", "Kamloops_Lillooet", "Southeast_Cranbrook",
              "Southeast_Invermere", "Southeast_Columbia", "Southeast_Arrow",
              "Southeast_Boundary", "Southeast_Kootenay", "Northwest_Nadina",
              "Northwest_Bulkley", "Northwest_Skeena", "Northwest_Cassiar",
              "Coastal_Fraser", "Coastal_Pemberton", "Coastal_SunshineCoast",
              "Coastal_SouthIsland", "Coastal_MidIsland", "Coastal_NorthIslandMidCoast"]

years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [22]:
#for zone in fire_zones:
#    for year in years:
#        for month in months:
#            fire_df1 = fire_data[fire_data['FIRE_ZONE'] == zone]
#            fire_df2 = fire_df1[fire_df1['FIRE_YEAR'] == year]
#            fire_df3 = fire_df2[fire_df2['FIRE_MONTH'] == month]
#            size_hectares = fire_df3[["SIZE_HA"]].sum()
#            fire_data = fire_data.drop(fire_df3.index)
#            new_row = pd.DataFrame({'FIRE_ZONE':zone, 'FIRE_YEAR':year, 'FIRE_MONTH':month, 'SIZE_HA':size_hectares}, index=[0])
#            fire_data = pd.concat([new_row,fire_data.loc[:]])

In [23]:
fire_df1 = fire_data[fire_data['FIRE_ZONE'] == "Cariboo_Quesnel"]
fire_df2 = fire_df1[fire_df1['FIRE_YEAR'] == 2009]
fire_df3 = fire_df2[fire_df2['FIRE_MONTH'] == "April"]
fire_data = fire_data.drop(fire_df3.index)
size = fire_df3[["SIZE_HA"]].sum()
print(size)
new_row = pd.DataFrame({'FIRE_ZONE':"Cariboo_Quesnel", 'FIRE_YEAR':[2009], 'FIRE_MONTH':"April", 'SIZE_HA':[size]})
print(new_row)
fire_data = pd.concat([new_row,fire_data.loc[:]])

SIZE_HA    35.7
dtype: float64
         FIRE_ZONE  FIRE_YEAR FIRE_MONTH                         SIZE_HA
0  Cariboo_Quesnel       2009      April  SIZE_HA    35.7
dtype: float64


In [10]:
#fire_data = fire_data.drop(fire_df3.index)

In [12]:
fire_data.to_csv('fire_data.csv')